In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from keras import *
from keras.models import *
from keras.optimizers import Adam
# plots in jupyter notebooks
%matplotlib inline
from modelDropOut import *
from metrics import  dice,IU,recall,precision,f1,specificity
from losses import bce_dice_loss,dice_loss
optim = Adam(lr = 1e-4)
img_size_target = 384
#model = UnetMobilnetV2( input_shape=(img_size_target, img_size_target, 3),encoder_freeze=False, encoder_weights='imagenet')
model = Unet(backbone_name='efficientnet-b5', input_shape=(img_size_target, img_size_target, 3), encoder_freeze=False, encoder_weights='imagenet')
#model = UnetResnet( backbone_name='ResNet101',img_size_target=img_size_target, input_shape=(img_size_target, img_size_target, 3),encoder_freeze=False, encoder_weights='None')
#model = UnetResnet( backbone_name='ResNet50',img_size_target=img_size_target, input_shape=(img_size_target, img_size_target, 3),encoder_freeze=False, encoder_weights='None')



Using TensorFlow backend.


Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


In [ ]:
model.summary()

In [ ]:
model.summary()

In [8]:
modelFolder = 'SaveModels/SSL-ClinicDB/B5'
filepathsave=os.path.join(modelFolder,'imagenet.h5')
model.load_weights(filepathsave)
model.compile(optimizer=optim, loss=bce_dice_loss, metrics=['acc',dice,IU,recall,precision,f1,specificity])

In [ ]:
from data import *
data_gen_args=dict()
test_generator = trainGenerator(2,'data/CVC-DB103-SSL/Suppervised/ColonDB/test1','image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [ ]:
results = model.predict_generator(test_generator,5,verbose=0)

In [ ]:
import cv2  
# importing os module  
import os  
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.transform as trans
import numpy as np
import skimage
from PIL import Image
# Image path
#ROOT = Path('/home/honglt/Unet') 
image_size = 384
image_path ='/home/honglt/Unet/data/CVC-DB103-SSL/Pretext/test/b5'  
for i in range(30):
    pred = np.reshape(results[i], (image_size, image_size))
    pred = (pred*255).astype(np.uint8)
    img = Image.fromarray(pred, 'L')
    img.save(os.path.join(image_path,'pred'+str(i) + '.png'))

In [ ]:
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.transform as trans
import numpy as np
import skimage
# plots in jupyter notebooks
%matplotlib inline
image_size=224
i=6
#sourceFiles_image = os.listdir("data/CVC-ColonDB/Image")
sourceFiles_image = os.listdir("data/CVC-DB103-SSL/Suppervised/ColonDB/test/image")
sourceFiles_label = os.listdir("data/CVC-DB103-SSL/Suppervised/ColonDB/test/label")
label_path="data/CVC-DB103-SSL/Suppervised/ColonDB/test/label"
image_path="data/CVC-DB103-SSL/Suppervised/ColonDB/test/image"
img_label = skimage.io.imread(os.path.join(label_path,sourceFiles_label[i]),as_gray = True)
img_label = trans.resize(img_label,(image_size, image_size))
img = io.imread(os.path.join(image_path,sourceFiles_image[i]))
fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.2)
ax = fig.add_subplot(2, 2, 1)
img_label[img_label > 0.5] = 1
img_label[img_label <= 0.5] = 0
ax.imshow(np.reshape(results[i], (image_size, image_size)), cmap="gray")
ax = fig.add_subplot(2, 2, 2)
ax.imshow(np.reshape(img_label, (image_size, image_size)), cmap="gray")
#img = trans.resize(image_path,(image_size, image_size,3))
ax = fig.add_subplot(2, 2, 3)
ax.imshow(img)
print(sourceFiles_image[i])
print(sourceFiles_label[i])

In [6]:
from data import *
data_gen_args=dict()
img_size_target = 384
Clinic_generator = trainGenerator(2,'data/CVC-ClinicDB','image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [ ]:
from data import *
data_gen_args=dict()
Kvasir_generator = trainGenerator(8,'data/Se2/test/Kvasir','image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [4]:
from data import *
from DropPixel import get_random_pixel_drop_img,get_random_pixel_drop_mask, get_addNoise_overimage
data_gen_args=dict()
img_size_target = 384
ETIS_generator = trainGenerator(2,'data/ETIS-Larib','image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [ ]:
from data import *
data_gen_args=dict()
Clinic_generator = trainGenerator(2,'CVC-ClinicDB','image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [ ]:
from data import *
data_gen_args=dict()
Kvasir_generator = trainGenerator(8,'data/Se2/test/Kvasir','image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [ ]:
from data import *
from DropPixel import get_random_pixel_drop_img,get_random_pixel_drop_mask, get_addNoise_overimage, get_random_drop
data_gen_args=dict()
data_gen_args_image = dict(preprocessing_function=get_addNoise_overimage(drop_scale=10))
data_gen_args_mask = dict(
                    preprocessing_function=get_random_pixel_drop_mask())
img_size_target = 384
#train_generator = Pretext_trainGenerator(2,csv_path,'train','train',data_gen_args_image,data_gen_args_mask,save_to_dir = None,image_size=img_size_target)
ETIS_generator_Noise = Pretext_trainGenerator(8,'data/ETIS-Larib','image','label',data_gen_args_image,data_gen_args_mask,save_to_dir = 'data/ETIS-Larib/test',image_size=img_size_target)

In [7]:
scores_val = model.evaluate_generator(Clinic_generator,612, verbose=0)

Found 612 images belonging to 1 classes.
Found 612 images belonging to 1 classes.


In [ ]:
scores_val = model.evaluate_generator(test_generator,30, verbose=0)

In [9]:
scores_val = model.evaluate_generator(ETIS_generator,196, verbose=0)

In [10]:
print("Score of:")
print("- val_acc:\t\t{}".format(scores_val[1]))
print("- val_dice:\t{}".format(scores_val[2]))
print("- val_iou:\t{}".format(scores_val[3]))
print("- val_recall:\t{}".format(scores_val[4]))
print("- val_precision:\t{}".format(scores_val[5]))
print("- val_f1:\t{}".format(scores_val[6]))
print("- specificity:\t{}".format(scores_val[7]))

Score of:
- val_acc:		0.9735047729039679
- val_dice:	0.7034811210188698
- val_iou:	0.5944900853340799
- val_recall:	0.774331250399047
- val_precision:	0.7127555320419523
- val_f1:	0.7070388186399882
- specificity:	0.9853563205320008


In [ ]:
def testGenerator(test_path,num_image = 300,target_size = (384,384),flag_multi_class = False,as_gray = False):
    sourceFiles1 = os.listdir(test_path)
    for i in range(len(sourceFiles1)):
        img = io.imread(os.path.join(test_path,sourceFiles1[i]),as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        #img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img

In [ ]:
ETIS_test=testGenerator('data/ETIS-Larib/image',num_image =196 ,target_size = (384,384),flag_multi_class = False,as_gray = False)

In [ ]:
results = model.predict_generator(ETIS_test,24,verbose=0)

In [ ]:
from data import testGenerator
testGene = testGenerator("data/CHAOS")
results = model.predict_generator(testGene,11,verbose=0)

In [ ]:
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.transform as trans
import numpy as np
import skimage
# plots in jupyter notebooks
%matplotlib inline
image_size=384
i=2
#sourceFiles_image = os.listdir("data/CVC-ColonDB/Image")
sourceFiles_image = os.listdir("data/ETIS-Larib/image")
sourceFiles_label = os.listdir("data/ETIS-Larib/label")
label_path="data/ETIS-Larib/label"
image_path="data/ETIS-Larib/image"
img_label = skimage.io.imread(os.path.join(label_path,sourceFiles_label[i]),as_gray = True)
img_label = trans.resize(img_label,(image_size, image_size))
img = io.imread(os.path.join(image_path,sourceFiles_image[i]))
fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.2)
ax = fig.add_subplot(2, 2, 1)
ax.imshow(np.reshape(results[i]*255, (image_size, image_size)), cmap="gray")
ax = fig.add_subplot(2, 2, 2)
ax.imshow(np.reshape(img_label, (image_size, image_size)), cmap="gray")
#img = trans.resize(image_path,(image_size, image_size,3))
ax = fig.add_subplot(2, 2, 3)
ax.imshow(img)
print(sourceFiles_image[i])
print(sourceFiles_label[i])

In [ ]:
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.transform as trans
import numpy as np
# plots in jupyter notebooks
%matplotlib inline
image_size=384
i=243
sourceFiles_image = os.listdir("data/CVC-ColonDB/Image")
sourceFiles_label = os.listdir("data/CVC-ColonDB/label")
label_path="data/CVC-ColonDB/label"
image_path="data/CVC-ColonDB/Image"
img_label = skimage.io.imread(os.path.join(label_path,sourceFiles_label[i]),as_gray = True)


In [12]:
sourceFiles_image = os.listdir("data/CVC-ClinicDB/image")
sourceFiles_label = os.listdir("data/ETIS-Larib/label")
print(len(sourceFiles_image))

613


In [13]:
for i in range(len(sourceFiles_image)):
    print(sourceFiles_image[i])

15.bmp
200.bmp
350.bmp
526.bmp
32.bmp
208.bmp
186.bmp
75.bmp
89.bmp
340.bmp
549.bmp
392.bmp
524.bmp
129.bmp
31.bmp
471.bmp
490.bmp
18.bmp
484.bmp
100.bmp
229.bmp
567.bmp
364.bmp
276.bmp
232.bmp
29.bmp
309.bmp
532.bmp
35.bmp
138.bmp
40.bmp
147.bmp
444.bmp
463.bmp
20.bmp
305.bmp
327.bmp
68.bmp
99.bmp
17.bmp
510.bmp
263.bmp
563.bmp
296.bmp
571.bmp
227.bmp
573.bmp
535.bmp
149.bmp
48.bmp
322.bmp
443.bmp
607.bmp
421.bmp
577.bmp
56.bmp
255.bmp
113.bmp
214.bmp
5.bmp
420.bmp
27.bmp
435.bmp
221.bmp
390.bmp
157.bmp
144.bmp
121.bmp
262.bmp
246.bmp
592.bmp
178.bmp
247.bmp
30.bmp
461.bmp
183.bmp
160.bmp
131.bmp
427.bmp
230.bmp
395.bmp
294.bmp
237.bmp
557.bmp
403.bmp
546.bmp
400.bmp
146.bmp
58.bmp
80.bmp
491.bmp
274.bmp
196.bmp
110.bmp
457.bmp
93.bmp
393.bmp
168.bmp
72.bmp
575.bmp
351.bmp
488.bmp
512.bmp
602.bmp
293.bmp
41.bmp
283.bmp
53.bmp
47.bmp
509.bmp
88.bmp
85.bmp
473.bmp
297.bmp
224.bmp
358.bmp
28.bmp
543.bmp
441.bmp
527.bmp
7.bmp
172.bmp
414.bmp
514.bmp
522.bmp
501.bmp
363.bmp
536.bmp
189.bmp

In [15]:
import os
if os.path.exists("data/CVC-ClinicDB/image/.ipynb_checkpoints"): 
    print("yes")
    os.remove("data/CVC-ClinicDB/image/.ipynb_checkpoints")
else:  print("The file does not exist")

yes


IsADirectoryError: [Errno 21] Is a directory: 'data/CVC-ClinicDB/image/.ipynb_checkpoints'